# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Harmony_1.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [2]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

print(lines[:10])

Dataset Stats
Roughly the number of unique words: 2519
Number of lines: 67068
Average number of words in each line: 1.1696636249776347

The lines 0 to 10:
<?xml version="1.0" encoding="UTF-8"?>
<museScore version="3.01">
  <programVersion>3.4.2</programVersion>
  <programRevision>148e43f</programRevision>
  <Score>
    <LayerTag id="0" tag="default"></LayerTag>
    <currentLayer>0</currentLayer>
    <Division>480</Division>
    <Style>
      <minSystemDistance>11.2</minSystemDistance>
['<?xml version="1.0" encoding="UTF-8"?>', '<museScore version="3.01">', '  <programVersion>3.4.2</programVersion>', '  <programRevision>148e43f</programRevision>', '  <Score>', '    <LayerTag id="0" tag="default"></LayerTag>', '    <currentLayer>0</currentLayer>', '    <Division>480</Division>', '    <Style>', '      <minSystemDistance>11.2</minSystemDistance>']


---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [3]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    counts = Counter(text)
    
    #trimmed_counts = [word for word in counts if counts[word] > 5]
    
    # Sort words per usage frequency
    vocabulary = sorted(counts, key=counts.get, reverse=True)
    
    #int_to_word = dict(enumerate(vocabulary, 1))
    
    # Build a dictionary
    vocab_to_int = {word: ii for ii, word in enumerate(vocabulary, 0)}
    
    # Build a dicitonary to transform integers to word
    int_to_word = {ii : word for word, ii in vocab_to_int.items()}
     
    # return tuple
    return (vocab_to_int, int_to_word)


# """
# DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
# """
#tests.test_create_lookup_tables(create_lookup_tables)


def create_sorted_lookup_tables(minVal, maxVal):

    # Build a dictionary
    vocab_to_int = {("<pitch>" + str(minVal + ii - 1) + "</pitch>" if ii != 0 else '<Rest>') : (ii if ii != 0 else 0)                    
                    for ii in range(0, maxVal + 1 - minVal)}
    
    # Build a dicitonary to transform integers to word
    int_to_word = {ii : word for word, ii in vocab_to_int.items()} 
    
    # return tuple
    return (vocab_to_int, int_to_word)

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    
    symbols_tokenisation = {
        ".":"_Period_",
        ",":"_Comma_",
        "\"":"Quotation_Mark",
        ";": "_Semicolon_",
        "!":"Exclamation_Mark",
        "?": "Question_Mark",
        "(": "Left_Parenthesis",
        ")": "Right_Parenthesis",
        "-": "_Dash_",
        "\n": "_Return_"
    }
      
    return symbols_tokenisation

# """
# DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
# """
#tests.test_tokenize(token_lookup)

## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
#helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

#Remove unnecessary text
#preliminary_text = [words.lstrip() for words in lines if (words.find("<durationType>") + words.find("<pitch>") + words.find("<tpc>") + words.find("<Rest>") + words.find("</Rest>")) > 0]
repetitive_preliminary_text = [words.lstrip() for words in lines if (words.find("<pitch>")) > 0]
print("repetitive_preliminary_text: ", len(repetitive_preliminary_text))

preliminary_text = [word for ii, word in enumerate(repetitive_preliminary_text) 
                    if (repetitive_preliminary_text[ii] != repetitive_preliminary_text[ii - 1]) & (ii != 0)]

int_preliminary_text = [int(word[7:-8]) for ii, word in enumerate(repetitive_preliminary_text) 
                    if (repetitive_preliminary_text[ii] != repetitive_preliminary_text[ii - 1]) & (ii != 0)]

print("preliminary_text: ", len(preliminary_text))

vocab_to_int, int_to_vocab = create_lookup_tables(preliminary_text)


#Sorted preliminary vocab
print(min(int_preliminary_text))
print(max(int_preliminary_text))
print("Sorted vocabulary: ", max(int_preliminary_text) - min(int_preliminary_text))

sorted_vocab_to_int, sorted_int_to_vocab = create_sorted_lookup_tables(min(int_preliminary_text), 
                                                                       max(int_preliminary_text) + 1)

print()
print("Unsorted vocabulary: ", len(vocab_to_int))
print(vocab_to_int)
print("***********")
print("Sorted vocabulary: ", len(vocab_to_int))
print(sorted_vocab_to_int)

repetitive_preliminary_text:  5477
preliminary_text:  5264
43
96
Sorted vocabulary:  53

Unsorted vocabulary:  53
{'<pitch>74</pitch>': 0, '<pitch>72</pitch>': 1, '<pitch>67</pitch>': 2, '<pitch>69</pitch>': 3, '<pitch>70</pitch>': 4, '<pitch>79</pitch>': 5, '<pitch>62</pitch>': 6, '<pitch>75</pitch>': 7, '<pitch>77</pitch>': 8, '<pitch>81</pitch>': 9, '<pitch>82</pitch>': 10, '<pitch>63</pitch>': 11, '<pitch>65</pitch>': 12, '<pitch>60</pitch>': 13, '<pitch>66</pitch>': 14, '<pitch>86</pitch>': 15, '<pitch>84</pitch>': 16, '<pitch>76</pitch>': 17, '<pitch>58</pitch>': 18, '<pitch>78</pitch>': 19, '<pitch>91</pitch>': 20, '<pitch>87</pitch>': 21, '<pitch>89</pitch>': 22, '<pitch>71</pitch>': 23, '<pitch>73</pitch>': 24, '<pitch>64</pitch>': 25, '<pitch>68</pitch>': 26, '<pitch>88</pitch>': 27, '<pitch>57</pitch>': 28, '<pitch>93</pitch>': 29, '<pitch>80</pitch>': 30, '<pitch>94</pitch>': 31, '<pitch>55</pitch>': 32, '<pitch>85</pitch>': 33, '<pitch>50</pitch>': 34, '<pitch>83</pitch>':

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

#int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()


int_text = [sorted_vocab_to_int[word] for word in preliminary_text]

print(int_text[:250])

[27, 28, 27, 25, 24, 25, 27, 20, 22, 24, 25, 23, 21, 20, 18, 16, 15, 16, 18, 20, 18, 16, 15, 13, 25, 27, 28, 27, 25, 24, 25, 27, 20, 22, 24, 25, 23, 21, 20, 18, 16, 15, 16, 25, 27, 16, 20, 16, 23, 18, 21, 20, 28, 23, 25, 20, 23, 21, 20, 18, 17, 18, 20, 21, 20, 18, 23, 18, 20, 28, 23, 25, 20, 23, 21, 30, 25, 27, 22, 25, 23, 22, 20, 19, 20, 32, 27, 29, 27, 25, 30, 25, 28, 27, 25, 23, 28, 23, 26, 25, 30, 25, 27, 22, 25, 24, 20, 25, 20, 21, 23, 18, 20, 21, 16, 18, 20, 15, 16, 15, 13, 12, 13, 15, 16, 15, 13, 20, 16, 13, 27, 32, 30, 28, 27, 25, 24, 25, 27, 20, 25, 23, 21, 20, 25, 30, 23, 28, 27, 28, 27, 25, 23, 28, 27, 25, 30, 28, 27, 32, 31, 32, 34, 27, 32, 31, 32, 30, 35, 33, 32, 37, 36, 37, 35, 33, 32, 33, 28, 27, 25, 33, 32, 37, 36, 37, 36, 37, 30, 28, 27, 35, 32, 37, 35, 33, 32, 33, 30, 35, 33, 32, 30, 32, 34, 35, 37, 35, 33, 32, 31, 27, 32, 30, 28, 27, 25, 23, 25, 27, 25, 23, 22, 20, 19, 28, 27, 25, 23, 22, 20, 29, 30, 32, 25, 38, 37, 35, 33, 32, 30, 36, 35, 33, 32, 27, 28, 30, 23, 28]

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

No GPU found. Please use a GPU to train your neural network.


## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [8]:
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from collections import Counter
import random
import numpy as np
import math
import random

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO: Implement function
    initial_words_counter = Counter(words)
    print("Initial vocabulary:", len(initial_words_counter))
    
    print("words[:10]: ", words[:10])
    
    #trimmed_words = [word for word in words if words[word] > 4 else print(word) for word in words]
    trimmed_words = [word for word in words]# if initial_words_counter[word] > 3]
    trimmed_words_complement = [word for word in words if initial_words_counter[word] <= 4]
    print("trimmed vocabulary:", trimmed_words[:10])
    threshold = 1e-5
    trimmed_counts = Counter(trimmed_words)
    
    print("length of trimmed_counts:", len(trimmed_counts))
    # discard some frequent words, according to the subsampling equation
    # create a new list of words for training

    total_len = len(trimmed_words)
    print("total length of the trimmed vocab:", total_len)
    #prob_list_func = lambda x : (1 - math.sqrt(threshold / ( x / total_len)))
    prob_list_func = lambda x : ( x / total_len)

    #WITHOUT SUBSAMPLING
    train_words = trimmed_words
    #SUBSAMPLING
    #train_words = [element for element in trimmed_words if ((1 - prob_list_func(trimmed_counts[element]) > 1.5 * random.random()) & (element <= 15)) | (element > 20)]
    print("train voacbulary:", train_words[:10])
        
    batch_size_total = batch_size * sequence_length
    # total number of batches we can make
    n_batches = len(train_words)//batch_size_total
    
    print("train_words length :", len(train_words))
    
    # Keep only enough characters to make full batches
    final_words = train_words[:n_batches * batch_size_total]
    output_final_text = final_words
   
    print("final_words length :", len(final_words))

    new_vocab = Counter(final_words)
    length_of_new_vocab = len(new_vocab)
    print("length_of_new_vocab :",  length_of_new_vocab)
    


#*************First Implementation needs to be tested************
#
#     target_data = np.zeros_like(words)
#     for index, data in enumerate(words):
#         if len(words) - (index + sequence_length) > 0:
#             target_data[index] = words[index + sequence_length]
#         else:
#             target_data[index] = words[index + sequence_length - len(words)]
#**********************************************************************
    
#    print("After :", len(words))


#*************Second Implementation needs to be tested************    

# Build target data
    
#     target_data = np.zeros((len(words)//sequence_length), dtype = int)
#     for index in range(len(words)//sequence_length):
#         if len(words) - ((index + 1) * sequence_length) > 0:
#             target_data[index] = words[(index  + 1) * sequence_length]
#         else:
#             target_data[index] = words[0]

#*************third Implementation needs to be tested************    

# Build target data
    step = sequence_length//sequence_length
    
    target_data = np.zeros((len(final_words)//step), dtype = int)
    print("length of target_data: ", len(final_words)//step) 


    new_words = np.zeros((len(final_words)//step,sequence_length), dtype = int)
    print("length of new_words: ", len(new_words)) 

    
    for index in range(len(final_words)//step):
        if len(final_words) - (index * step + sequence_length) > 0:
            target_data[index] = final_words[index * step + sequence_length]
            new_words[index] = final_words[index * step : index * step + sequence_length]
        else:
            target_data[index] = final_words[(index * step + sequence_length) - len(final_words) + sequence_length]
            new_words[index] = final_words[(index * step + sequence_length) - len(final_words) : (index * step + sequence_length) - len(final_words) + sequence_length]

    final_words = np.array(new_words)
    target_data = np.array(target_data)
    final_words = final_words.reshape([-1, sequence_length])
    
    print("final_words.shape:", final_words.shape)
    print("target_data.shape:", target_data.shape)
    
    train_data = TensorDataset(torch.from_numpy(final_words), torch.from_numpy(target_data))
    
    # Build dataloader
    train_loader = DataLoader(train_data, shuffle = True, batch_size = batch_size)
    
    # return a dataloader
    return train_loader, length_of_new_vocab, output_final_text

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [9]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()
        # TODO: Implement function
        
        # set class variables
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=dropout, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(dropout)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        # define model layers
    
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        # TODO: Implement function   
        batch_size = nn_input.size(0)

        # embeddings and lstm_out
        nn_input = nn_input.long()
        embeds = self.embedding(nn_input)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        #out = nn.ReLU()
        output = self.fc(out)
       
        # reshape to be batch_size first
        output = output.view(batch_size, -1, self.output_size)
        output = output[:, -1] # get last batch of labels
        
        # return one batch of output word scores and the hidden state
        return output, hidden
   
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        
        # initialize hidden state with zero weights, and move to GPU if available
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [10]:
i = 0

In [11]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    global i, loss
    # TODO: Implement Function
    
    # move data to GPU, if available
    if(train_on_gpu): 
        rnn.cuda()
        inp, target = inp.cuda(), target.cuda()

    # perform backpropagation and optimization
    # zero accumulated gradients
    rnn.zero_grad()

    # Creating new variables for the hidden state, otherwise
    # we'd backprop through the entire training history
    h = tuple([each.data for each in hidden])

    # get the output and hidden layer from the model
    output, h = rnn(inp, hidden)

    # calculate the loss and perform backprop
    loss = criterion(output.squeeze(), target.long())
    
#     if i == 0:
#         loss.backward(retain_graph=True)
#         i = i + 1
#         print(i)
#     else:
#         loss.backward()
    
    #loss.backward(retain_graph=True)
    loss.backward()
    # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
    #nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    
    # Update weights
    optimizer.step()
   
    output_loss = loss.item()
    
    # return the loss over a batch and the hidden state produced by our model
    return output_loss, hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in range(1, n_epochs + 1):
        
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            # record loss
            batch_losses.append(loss)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []
#******** New Code to save model every 10 epochs *************               
        if (epoch_i % 1 == 0) & (epoch_i != 0):
            # saving the trained model
            helper.save_model('./save/trained_rnn' + str(epoch_i), rnn)
            print('Model Trained and Saved')

        if np.average(batch_losses) < 0.5:
            # saving the trained model
            #helper.save_model('./save/trained_rnn', rnn)
            #print('Model Trained and Saved')
            return rnn
#******** End of new code ************************************

    # returns a trained rnn
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [13]:
# Data params
# Sequence Length
sequence_length =  5 # of words in a sequence
# Batch Size
batch_size = 32

# data loader - do not change
train_loader, vocab_length, new_trimmed_text = batch_data(int_text, sequence_length, batch_size)

print()
print("vocab length: ", len(sorted_int_to_vocab))
print()
print("new trimmed text: ", len(new_trimmed_text))

Initial vocabulary: 53
words[:10]:  [27, 28, 27, 25, 24, 25, 27, 20, 22, 24]
trimmed vocabulary: [27, 28, 27, 25, 24, 25, 27, 20, 22, 24]
length of trimmed_counts: 53
total length of the trimmed vocab: 5264
train voacbulary: [27, 28, 27, 25, 24, 25, 27, 20, 22, 24]
train_words length : 5264
final_words length : 5120
length_of_new_vocab : 53
length of target_data:  5120
length of new_words:  5120
final_words.shape: (5120, 5)
target_data.shape: (5120,)

vocab length:  55

new trimmed text:  5120


In [14]:
# Training parameters
# Number of Epochs
num_epochs = 20
# Learning Rate
learning_rate = 0.005 * batch_size / 256

# Model parameters
# Vocab size
vocab_size = len(sorted_int_to_vocab)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 15
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 30
i = 0

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.0)
if train_on_gpu:
    rnn.cuda()

print(rnn)

#************ Use pretrained rnn*************

#trained_rnn = helper.load_model('./saved_models/20200418/trained_rnn')
#trained_rnn = helper.load_model('./trained_rnn')
#rnn = trained_rnn
#********************************************

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

RNN(
  (embedding): Embedding(55, 15)
  (lstm): LSTM(15, 256, num_layers=3, batch_first=True)
  (dropout): Dropout(p=0.0, inplace=False)
  (fc): Linear(in_features=256, out_features=55, bias=True)
  (sig): Sigmoid()
)
Training for 20 epoch(s)...
Epoch:    1/20    Loss: 3.624995231628418

Epoch:    1/20    Loss: 3.280479073524475

Epoch:    1/20    Loss: 3.1622747580210366

Epoch:    1/20    Loss: 3.064320182800293

Epoch:    1/20    Loss: 2.818873373667399



/home/achilleas/anaconda3/envs/deep-learning/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/achilleas/anaconda3/envs/deep-learning/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/achilleas/anaconda3/envs/deep-learning/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/achilleas/anaconda3/envs/deep-learning/lib/python3.7/site-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Dropout.

Model Trained and Saved
Epoch:    2/20    Loss: 2.74114243388176

Epoch:    2/20    Loss: 2.6992408831914267

Epoch:    2/20    Loss: 2.5941655158996584

Epoch:    2/20    Loss: 2.6774325450261434

Epoch:    2/20    Loss: 2.6239514350891113

Model Trained and Saved
Epoch:    3/20    Loss: 2.531173312664032

Epoch:    3/20    Loss: 2.5513588587443032

Epoch:    3/20    Loss: 2.5328234990437823

Epoch:    3/20    Loss: 2.5341207504272463

Epoch:    3/20    Loss: 2.4949018796284994

Model Trained and Saved
Epoch:    4/20    Loss: 2.459094798564911

Epoch:    4/20    Loss: 2.4301995356877644

Epoch:    4/20    Loss: 2.4330027103424072

Epoch:    4/20    Loss: 2.472556495666504

Epoch:    4/20    Loss: 2.40882187684377

Model Trained and Saved
Epoch:    5/20    Loss: 2.4317667543888093

Epoch:    5/20    Loss: 2.3500096718470256

Epoch:    5/20    Loss: 2.265153161684672

Epoch:    5/20    Loss: 2.345324675242106

Epoch:    5/20    Loss: 2.3068598031997682

Model Trained and Saved
Epoch:   

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_intDONOTUSE, int_to_vocabDONOTUSE, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    print(current_seq.shape)
    
    predicted = [int_to_vocab[prime_id]]
    
    for ii in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [18]:
# run the cell multiple times to get different results!
gen_length = 500 # modify the length to your preference
prime_word = '<pitch>67</pitch>' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
#pad_word = helper.SPECIAL_WORDS['PADDING']

# Update padding word to use rest
pad_word = "<pitch>62</pitch>"

# generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
generated_script = generate(trained_rnn, sorted_vocab_to_int[prime_word], sorted_int_to_vocab, token_dict, sorted_vocab_to_int[pad_word], gen_length)
#print(generated_script)

(1, 5)


In [19]:
generated_melody_list = generated_script.split('\n')
generated_melody = '\n'.join(generated_melody_list)
# save script to a text file
f =  open("generated_melody.txt","w")
f.write(generated_melody)
f.close()

In [20]:

# Create a reusable script
preliminary_music_script = generated_script.split()
print(len(preliminary_music_script))
new_music_script = []
new_music_script.append("<Measure>")
new_music_script.append("<voice>")
lcl_measure = 1/8
duration_dict = {"<durationType>16th</durationType>" : 1/16,
                "<durationType>eighth</durationType>" : 1/8,
                "<durationType>quarter</durationType>" : 1/4,
                "<durationType>half</durationType>" : 1/2,
                "<durationType>whole</durationType>" : 1}

CloseMeasure = False



for index in range(len(preliminary_music_script) - 2):
    new_index = index
#     duration_dict[preliminary_music_script[new_index]]
    if lcl_measure == 1:
        lcl_measure = 0
        CloseMeasure = True
  
    new_music_script.append("<Chord>")
    new_music_script.append("<durationType>eighth</durationType>")
    #new_music_script.append(preliminary_music_script[new_index])
    new_music_script.append("<Note>")
    #new_music_script.append(preliminary_music_script[new_index + 1])
    #new_music_script.append(preliminary_music_script[new_index + 2])
    new_music_script.append(preliminary_music_script[new_index])
    new_music_script.append("</Note>")
    new_music_script.append("</Chord>")
    
    lcl_measure += 1/8
    
    if CloseMeasure == True:
        new_music_script.append("</voice>")
        new_music_script.append("</Measure>")
        new_music_script.append("<Measure>")
        new_music_script.append("<voice>")
        CloseMeasure = False

print("music script length ", len(new_music_script))  
music_script = ' '.join(new_music_script)
#print(music_script)

for ii in range(len(new_music_script)):
    print(new_music_script[ii])

501
music script length  3244
<Measure>
<voice>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>67</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>62</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>62</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>69</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>70</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>69</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>67</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>69</pitch>
</Note>
</Chord>
</voice>
</Measure>
<Measure>
<voice>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>62</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>74</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pi

<durationType>eighth</durationType>
<Note>
<pitch>75</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>72</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>69</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>71</pitch>
</Note>
</Chord>
</voice>
</Measure>
<Measure>
<voice>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>65</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>64</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>60</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>58</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>57</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>64</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>67</pitch>
</Note>
</Chord>
<Chord>
<durationType>ei

</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>67</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>74</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>72</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>74</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>66</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>74</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>72</pitch>
</Note>
</Chord>
</voice>
</Measure>
<Measure>
<voice>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>74</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>67</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>74</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>72</pitch>
</Note>
</Chord>

<durationType>eighth</durationType>
<Note>
<pitch>70</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>72</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>68</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>72</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>70</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>72</pitch>
</Note>
</Chord>
</voice>
</Measure>
<Measure>
<voice>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>77</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>72</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>70</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>72</pitch>
</Note>
</Chord>
<Chord>
<durationType>eighth</durationType>
<Note>
<pitch>68</pitch>
</Note>
</Chord>
<Chord>
<durationType>ei

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [21]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(music_script)
f.close()